In [65]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
import string
from nltk import pos_tag
from nltk.corpus import stopwords
# nltk.download('stopwords')

from mi_helper import *

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from nltk.corpus import wordnet
import re

from numpy import array 

In [2]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
print(dataset["feedback"].value_counts())
print(dataset.shape)
# data_test = pd.read_csv("test_set.csv", sep = "\t")
# data_test.drop("Unnamed: 0", axis = 1, inplace = True)
# dataset = pd.concat([data_test, dataset])
# dataset.dropna(inplace=True)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3 stelle
# dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 4 stelle
print(dataset.shape)
print(dataset["feedback"].value_counts())

1    2893
0     257
Name: feedback, dtype: int64
(3150, 5)
(2998, 5)
1    2741
0     257
Name: feedback, dtype: int64


In [3]:
custom_stopwords = ['echo', 'alexa', "alexia", 'dot', "star", 'amazon', 'prime', '2nd', 'generation', "fire", "stick", "firestick", "skype", "facetime", '1st', '3rd', '4th', '5th', "hub", "hulu", 'google', 'netflix', "sony", 'youtube', 'philip', 'tp-link', 'fourth', 'roku', "siri", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "...", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']


def negation_handler(sentence):	
    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't", "no", "without"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [4]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza. Ritorna la colonna delle recensioni."""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    detokenizer = TreebankWordDetokenizer()

    tokenized_reviews = []
    sent_tokenized_reviews = []
    for review in list_of_text: #pulisce le recensioni
        review = re.sub(r'\d+', '', review)
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = negation_handler(tokens)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords]
        sent_tokenized_reviews.append([w for w, pos in clean_tokens])
        tokenized_reviews.append(detokenizer.detokenize([w for w, pos in clean_tokens]))
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [5]:
tokenized_reviews, sentences = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, pos_filter=False, pos_list = pos_list)

dataset["verified_reviews"] = tokenized_reviews

total number of tokens extracted are: 3171


In [126]:
# CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


dataset = create_balanced_dataset(dataset)

1    257
0    257
Name: feedback, dtype: int64


In [6]:
dataset

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,love,1
1,5,31-Jul-18,Charcoal Fabric,love,1
2,4,31-Jul-18,Walnut Finish,sometimes play game answer question correctly ...,1
3,5,31-Jul-18,Charcoal Fabric,lot fun thing old learns dinosaur control ligh...,1
4,5,31-Jul-18,Charcoal Fabric,music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,perfect kid adult everyone,1
3146,5,30-Jul-18,Black Dot,listen music search location check time look w...,1
3147,5,30-Jul-18,Black Dot,love thing run entire home light thermostat fr...,1
3148,5,30-Jul-18,White Dot,complaint sound qualityn't great mostly use co...,1


In [7]:
tokenizer = nltk.tokenize.TweetTokenizer()
cv = CountVectorizer(stop_words="english", ngram_range=(1, 3), tokenizer=tokenizer.tokenize, min_df = 4) #count == frequenza
text_counts = cv.fit_transform(dataset["verified_reviews"]) 
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset["feedback"], test_size = 0.20, random_state=10) #divisione in train e test

In [8]:
text_counts

<2998x1786 sparse matrix of type '<class 'numpy.int64'>'
	with 29892 stored elements in Compressed Sparse Row format>

In [9]:
len(cv.vocabulary_)

1786

In [10]:
cv.vocabulary_ # A mapping of terms to feature indices.

{'love': 869,
 'play': 1113,
 'game': 549,
 'answer': 68,
 'question': 1215,
 'correctly': 293,
 'say': 1314,
 'wrong': 1776,
 'like': 792,
 'able': 11,
 'turn': 1593,
 'light': 786,
 'away': 111,
 'home': 659,
 'play game': 1116,
 'answer question': 69,
 'like able': 794,
 'turn light': 1595,
 'lot': 863,
 'fun': 537,
 'thing': 1535,
 'old': 1060,
 'control': 281,
 'nice': 1041,
 'sound': 1422,
 'music': 979,
 'lot fun': 864,
 'control light': 282,
 'game like': 550,
 'nice sound': 1044,
 'sound play': 1433,
 'play music': 1120,
 'receive': 1250,
 'gift': 554,
 'need': 1023,
 'bluetooth': 155,
 'easily': 407,
 'smart': 1394,
 'speaker': 1448,
 'wait': 1687,
 'need bluetooth': 1024,
 'bluetooth play': 157,
 'music easily': 990,
 'smart speaker': 1403,
 'bluetooth play music': 158,
 'think': 1549,
 'purchase': 1202,
 'work': 1739,
 'room': 1294,
 'house': 678,
 'really': 1237,
 'feature': 507,
 'offer': 1057,
 'room house': 1296,
 'really like': 1243,
 'like feature': 800,
 'music contr

In [11]:
#get_feature_names_out([input_features])
#Get output feature names for transformation.

len(cv.get_feature_names_out())

1786

In [12]:
X_train[0,:] # prima recensione

<1x1786 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [13]:
print(X_train[0,:])

  (0, 869)	1
  (0, 1422)	1
  (0, 1549)	1
  (0, 1739)	1
  (0, 584)	1
  (0, 1183)	1
  (0, 319)	1
  (0, 559)	1
  (0, 517)	1
  (0, 1210)	1
  (0, 1435)	1
  (0, 929)	1
  (0, 1747)	1
  (0, 1092)	1
  (0, 1184)	1
  (0, 1550)	1
  (0, 1551)	1
  (0, 614)	1
  (0, 1753)	1
  (0, 930)	1


In [14]:
cv.inverse_transform(X_train[0,:]) #0 ==> prima recensione

[array(['love', 'sound', 'think', 'work', 'great', 'problem', 'day',
        'good', 'fine', 'quality', 'sound quality', 'love work',
        'work great', 'perfect', 'problem day', 'think sound',
        'think sound quality', 'great problem', 'work great problem',
        'love work great'], dtype='<U25')]

In [15]:
for feat,freq in zip(cv.inverse_transform(X_train[0,:])[0],X_train[0,:].data):
    print(feat,freq)

love 1
sound 1
think 1
work 1
great 1
problem 1
day 1
good 1
fine 1
quality 1
sound quality 1
love work 1
work great 1
perfect 1
problem day 1
think sound 1
think sound quality 1
great problem 1
work great problem 1
love work great 1


In [17]:
#For classification we'll set 'chi2'  method as a scoring function. The target number of features is defined by k parameter
select = SelectKBest(chi2, k=1700)  # feature selection
# select = SelectKBest(chi2, k="all")  # feature selection for balanced with pos filter
select.fit(X_train,Y_train)
X_train_sel = select.transform(X_train)
X_test_sel = select.transform(X_test)

In [18]:
#We've selected 3 best features in x data. To identify the selected features we use get_support() function and filter out them from the features name list. The z object contains selected x data
filter = select.get_support() #filtra le parole selezionando quelle contenenti nei k 5000
sum(filter)

1700

In [19]:
X_train_sel

<2398x1700 sparse matrix of type '<class 'numpy.int64'>'
	with 21051 stored elements in Compressed Sparse Row format>

In [20]:
print(X_test_sel[0,:])

  (0, 1647)	1


In [21]:
print(cv.inverse_transform(select.inverse_transform(X_train_sel[0,:]))) #applichi l'inverse_transform sui vettori che poi hai utilizzato con il Select

[array(['day', 'fine', 'great', 'great problem', 'love', 'love work',
       'love work great', 'perfect', 'problem', 'problem day', 'quality',
       'sound', 'sound quality', 'think', 'think sound',
       'think sound quality', 'work', 'work great', 'work great problem'],
      dtype='<U25')]


In [22]:
tfidf = TfidfTransformer()  # weighting
tfidf.fit(X_train_sel)
X_train_vec = tfidf.transform(X_train_sel)
X_test_vec =tfidf.transform(X_test_sel)

In [23]:
print(X_train_vec[0,:])

  (0, 1668)	0.31117854321381166
  (0, 1662)	0.1715447057463448
  (0, 1655)	0.11835575615267203
  (0, 1474)	0.32085844143647685
  (0, 1473)	0.31117854321381166
  (0, 1472)	0.18776725426600765
  (0, 1363)	0.19859719691328095
  (0, 1350)	0.135944707104709
  (0, 1151)	0.17669749878983323
  (0, 1127)	0.31117854321381166
  (0, 1126)	0.20338406149517235
  (0, 1039)	0.19991183359890047
  (0, 885)	0.3032694904844399
  (0, 884)	0.2635210820062406
  (0, 825)	0.0953492484657902
  (0, 585)	0.31117854321381166
  (0, 556)	0.11071910425957383
  (0, 496)	0.22097301244720932
  (0, 306)	0.1701783033139707


In [24]:
cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))

[array(['day', 'fine', 'great', 'great problem', 'love', 'love work',
        'love work great', 'perfect', 'problem', 'problem day', 'quality',
        'sound', 'sound quality', 'think', 'think sound',
        'think sound quality', 'work', 'work great', 'work great problem'],
       dtype='<U25')]

In [25]:
for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[0,:]))[0],X_train_vec[0,:].data,X_train_sel[1,:].data):
    print(feat,round(weight,4),freq)

day 0.3112 1
fine 0.1715 1
great 0.1184 1
great problem 0.3209 1


In [26]:
# create a dataframe with words, tf_idf score and freq
total_scores = []

for i in range(X_train_vec.shape[0]):
    for feat,weight,freq in zip(cv.inverse_transform(select.inverse_transform(X_train_vec[i,:]))[0],X_train_vec[i,:].data,X_train_sel[i,:].data):
        total_scores.append((feat,round(weight,4),freq))

words = []
tf_idf_score = []
freq = []
for i in range(len(total_scores)):
    words.append(total_scores[i][0])
    tf_idf_score.append(total_scores[i][1])
    freq.append(total_scores[i][2])

scores_df = pd.DataFrame()
scores_df["words"] = words
scores_df["tf_idf_score"] = tf_idf_score
scores_df["freq"] = freq

In [27]:
scores_df.sort_values(by = "tf_idf_score", ascending = False).head(50) # sort and show the df

,words,tf_idf_score,freq
10305,friend,1.0,1
5434,terrific,1.0,1
1927,buy,1.0,1
8519,awesome,1.0,1
1980,sound,1.0,1
18069,apps,1.0,1
1999,love,1.0,1
5599,excellent,1.0,1
16536,love,1.0,1
8638,gift,1.0,1


In [28]:
X_train_vec

<2398x1700 sparse matrix of type '<class 'numpy.float64'>'
	with 21051 stored elements in Compressed Sparse Row format>

In [70]:
svm = LinearSVC(max_iter=3000, class_weight = "balanced")  # linear svm with default parameters
svm_clf = svm.fit(X_train_vec,Y_train)
predictions = svm_clf.predict(X_test_vec)

In [71]:
len(predictions), sum(predictions)

(600, 517)

In [72]:
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.43      0.75      0.55        48
           1       0.98      0.91      0.94       552

    accuracy                           0.90       600
   macro avg       0.71      0.83      0.75       600
weighted avg       0.93      0.90      0.91       600



In [50]:
feature_names = cv.get_feature_names_out()
feature_names

array(["'", "' t", "' ve", ..., 'yesterday', 'young', 'zigbee'],
      dtype=object)

In [51]:
print(select.scores_)

[9.52249285 2.28519792 0.14400952 ... 0.28643216 0.28643216 4.2845808 ]


In [52]:
feature_names = cv.get_feature_names_out()
feats_w_score = list()
for index,(selected,score) in enumerate(zip(filter,select.scores_)):
    feats_w_score.append((score,selected,feature_names[index]))
feats_w_score = sorted(feats_w_score, reverse = True)
len(feats_w_score)

1786

In [53]:
feats_w_score[:10] # this contains k score, wether it is a selcted feature and the word

[(110.44021158423698, True, 'return'),
 (95.07021029258901, True, 'try'),
 (90.18842924633488, True, 'device'),
 (63.63101724159645, True, 'love'),
 (58.0799211367844, True, 'month'),
 (52.36842105263156, True, 'horrible'),
 (52.36842105263156, True, 'disappointing'),
 (50.326834674626255, True, 'idle'),
 (47.206982279820146, True, 'half'),
 (46.058004704965256, True, 'disappointed')]

In [54]:
type(svm)

sklearn.svm._classes.LinearSVC

In [55]:
len(svm.coef_)

1

In [56]:
feats_w_classifier_weight = list()
for index,weight in enumerate(select.inverse_transform(svm.coef_)[0]):
    if weight!=0:
        feats_w_classifier_weight.append((weight,feature_names[index]))
feats_w_classifier_weight = sorted(feats_w_classifier_weight)
len(feats_w_classifier_weight)

1620

In [57]:
feats_w_classifier_weight[-100:] #features positive

[(0.7024690554838207, 'work perfectly'),
 (0.70442634628752, 'easy use'),
 (0.7083950564493117, 'use time'),
 (0.7095476363058358, 'date'),
 (0.7107039174528453, 'happy purchase'),
 (0.7122502154565195, 'country'),
 (0.7280068102228819, 'speaker good'),
 (0.7310026185857919, 'satisfy'),
 (0.7329750132982532, 'stream'),
 (0.7339295436318084, 'certain'),
 (0.7340326420611117, 'buy day'),
 (0.7353568806063581, 'misuse'),
 (0.7378318630146913, 'way use'),
 (0.7440028879767369, 'exactly'),
 (0.7510536125168995, 'ask'),
 (0.7546910056513384, 'include'),
 (0.7564694286997675, 'discover'),
 (0.7642450248184152, 'perfectly'),
 (0.7662526586347743, 'totally'),
 (0.7733759839288028, 'work need'),
 (0.7843013519156149, 'voice command'),
 (0.7845407227194484, 'awhile'),
 (0.7875650545934455, 'bedside'),
 (0.7900160480069929, 'easy set'),
 (0.7915766046598365, 'deliver'),
 (0.792162758635722, 'like new'),
 (0.79237616401865, 'skill'),
 (0.793034183626522, 'power'),
 (0.7954210410726554, 'terrific'),

In [58]:
feats_w_classifier_weight[:100] #features negative

[(-1.8707146132304122, 'send'),
 (-1.8664118143325987, 'return'),
 (-1.818096940068467, 'poor'),
 (-1.7437929950825455, 'adapter'),
 (-1.716904266496563, 'party'),
 (-1.6936926293038628, 'terrible'),
 (-1.663887028710954, 'realize'),
 (-1.4345700573714026, 'half'),
 (-1.3938885178850724, 'stop work'),
 (-1.3424887984184286, 'useless'),
 (-1.3037327641432097, 'hue'),
 (-1.2860356817429683, 'dont'),
 (-1.2778613304090387, 'snooze'),
 (-1.259028449540562, 'horrible'),
 (-1.236367616535872, '.'),
 (-1.2345130852638038, 'product great'),
 (-1.2307796498658212, 'speak'),
 (-1.22962593702266, 'past'),
 (-1.2125750610205996, 'hardly'),
 (-1.201907596736826, 'bass'),
 (-1.196774709042626, 'sonos'),
 (-1.181275860433503, 'idle'),
 (-1.174414759417275, 'connect phone'),
 (-1.1741678550437038, 'reason'),
 (-1.1559227732066741, 'stop'),
 (-1.1531142750918355, 'unsure'),
 (-1.142994384265778, 'refurbished'),
 (-1.138868534331435, 'pair'),
 (-1.1092023330821914, 'try'),
 (-1.0894128579192723, 'unless

In [59]:
df_scores = pd.DataFrame()
scores = []
words = []
for entry in feats_w_classifier_weight:
    scores.append(entry[0])
    words.append(entry[1])

df_scores["scores"] = scores
df_scores["words"] = words

df_scores.sort_values(by = "scores", ascending = False).head(15) # use tail for seeing the positive words

,scores,words
1619,3.830469,love
1618,2.690722,great
1617,2.572085,easy
1616,1.663627,enjoy
1615,1.592203,fun
1614,1.549162,far
1613,1.545778,best
1612,1.402629,room
1611,1.332568,perfect
1610,1.328555,option


In [60]:
stringa = "the product is disappointing, not good at all"
clean = tokenize_list_of_text([stringa])[0]
clean

total number of tokens extracted are: 5


['the product disappointing evil all']

In [61]:
vector = cv.transform(clean)
vector = select.transform(vector)
vector = tfidf.transform(vector)
predicted = svm.predict(vector)
print(predicted)

[0]


In [62]:
MNB = MultinomialNB()  # MNB with default parameters
MNB_clsf = MNB.fit(X_train_vec,Y_train)
predictions = MNB_clsf.predict(X_test_vec)
print(metrics.classification_report(predictions, Y_test))

              precision    recall  f1-score   support

           0       0.06      1.00      0.12         3
           1       1.00      0.92      0.96       597

    accuracy                           0.93       600
   macro avg       0.53      0.96      0.54       600
weighted avg       1.00      0.93      0.96       600



In [63]:
len(Y_test)

600

In [67]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

grid = GridSearchCV(SVC(), param_grid, refit ='f1_macro', scoring="f1_macro", cv=kfold, verbose=3)
grid.fit(X_train_vec, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.723 total time=   0.3s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.687 total time=   0.2s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.672 total time=   0.3s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.702 total time=   0.2s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.663 total time=   0.3s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.690 total time=   0.2s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.713 total time=   0.2s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.713 total time=   0.2s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.714 total time=   0.2s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             refit='f1_macro', scoring='f1_macro', verbose=3)

In [68]:
print(grid.best_params_)

{'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'linear'}


In [69]:
grid_predictions = grid.predict(X_test_vec)

print(metrics.classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.48      0.67      0.56        48
           1       0.97      0.94      0.95       552

    accuracy                           0.92       600
   macro avg       0.73      0.80      0.76       600
weighted avg       0.93      0.92      0.92       600

